# openAIのEmbeddingsAPIを使って記事タイトルをベクトル化して類似度を計算する
ブログ：https://techblog.gmo-ap.jp/2023/06/22/embeddings_api_calc_sentence_similarity/

# 初期設定

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import torch
import openai
openai.api_key = "XXXX"

# 文のベクトル化



In [4]:
def sentence_to_vector(sentence):
  res = openai.Embedding.create(
        model='text-embedding-ada-002',
        input=sentence
    )

  return torch.tensor(res['data'][0]["embedding"])

In [5]:
sentence = "AIが医療診断の精度を大幅に向上させる"
TARGET_VECTOR = sentence_to_vector(sentence)

In [6]:
TARGET_VECTOR

tensor([-0.0055,  0.0198,  0.0074,  ..., -0.0054, -0.0081, -0.0051])

# 記事タイトルの類似度計算

## 記事データの生成

In [7]:
import pandas as pd
article_df = pd.DataFrame({"title": ["AI技術の進展により、医療診断の正確性が飛躍的に向上",
                                     "サイバーセキュリティの脅威がますます増加、対策の重要性が高まる",
                                     "最新AIテクノロジーが医療診断に革命をもたらす",
                                     "テクノロジーがもたらす教育の変革：デジタル学習の普及と課題",
                                     "AIの登場により、医師の診断能力が向上する可能性",
                                     "宇宙探査の新たな時代：民間企業の参入と宇宙開発の可能性",
                                     "AIが医療診断のエラーレートを劇的に減少させる",
                                     "人工知能が医療診断における新たな道を開く",
                                     "持続可能なエネルギーの未来：再生可能エネルギーの普及と課題",
                                     "都市の未来：スマートシティの実現と利益、課題"]})

## 類似度算出

In [8]:
def calc_similarity(sentence):
  sentence_vector = sentence_to_vector(sentence)

  score = torch.nn.functional.cosine_similarity(TARGET_VECTOR, sentence_vector, dim=0).detach().numpy().copy()
  return score

In [9]:
article_df["similarity"] = article_df["title"].apply(lambda title: calc_similarity(title))

## 結果確認

In [13]:
print(f"元記事タイトル：{sentence}\n")
article_df.sort_values('similarity', ascending=False)

元記事タイトル：AIが医療診断の精度を大幅に向上させる



,title,similarity
0,AI技術の進展により、医療診断の正確性が飛躍的に向上,0.960818
4,AIの登場により、医師の診断能力が向上する可能性,0.951661
6,AIが医療診断のエラーレートを劇的に減少させる,0.941512
2,最新AIテクノロジーが医療診断に革命をもたらす,0.936800
7,人工知能が医療診断における新たな道を開く,0.925921
1,サイバーセキュリティの脅威がますます増加、対策の重要性が高まる,0.819197
3,テクノロジーがもたらす教育の変革：デジタル学習の普及と課題,0.811262
9,都市の未来：スマートシティの実現と利益、課題,0.802044
5,宇宙探査の新たな時代：民間企業の参入と宇宙開発の可能性,0.796668
8,持続可能なエネルギーの未来：再生可能エネルギーの普及と課題,0.767106
